#  Cardio Project Deploy

### Steps:

- Create a python file to the Cardio Class.
- Create a python file to the API handler.
- Replace the paths for your computer.
- Save the model.
- Use the section API Tester to input the raw data and make the prediction.

In [141]:
# The model was fitted only inside functions, so it's necessary to fit it again to save it.
LG = LGBMClassifier()
LG.fit(X, y)

# Save the model LGBM
import pickle
pickle.dump(LG, open('C:/Users/Usuario/Desktop/Data_Science/PROJETOS_DATA_SCIENCE/Projeto_Cardio_Disease/deploy/model_LGBM.pkl', 'wb'))

## Cardio Class

In [142]:
import pandas as pd

class Cardio():
  
    def data_preparation(self, df):                    # cleaning, preparation, feature engineering and enconding
        
        # Age in years
        df['age_years'] = df['age'] / 360

        # Drop the columns 'id' and 'age' and target 'cardio'
        df.drop(['id','age', 'cardio'], axis='columns', inplace=True)

        # Creating feature 'IMC'
        df1 = df.copy()
        df1['IMC'] = df1['weight']/((df1['height']/100) * (df1['height']/100))

        # Creating feature 'dwarfism'
        df1['dwarfism'] = [1 if value < 145 else 0 for value in df1['height']]

        # Creating feature 'blood_pressure' manually as a dummie
        df1['blood_pressure_1'] = df1[['ap_hi', 'ap_lo']].apply(lambda x: 1 if (x['ap_hi'] < 90) & (x['ap_lo'] < 60) else 0, axis=1)
        df1['blood_pressure_2'] = df1[['ap_hi', 'ap_lo']].apply(lambda x: 1 if (90 <= x['ap_hi'] < 140) & (60 <= x['ap_lo'] < 90) else 0, axis=1)
        df1['blood_pressure_3'] = df1[['ap_hi', 'ap_lo']].apply(lambda x: 1 if (140 <= x['ap_hi'] < 160) & (90 <= x['ap_lo'] < 100) else 0, axis=1)
        df1['blood_pressure_4'] = df1[['ap_hi', 'ap_lo']].apply(lambda x: 1 if (x['ap_hi'] >= 160) & (x['ap_lo'] >= 100) else 0, axis=1)
        df1['blood_pressure_5'] = df1[['ap_hi', 'ap_lo']].apply(lambda x: 0 if (x['ap_hi'] < 90) & (x['ap_lo'] < 60) else
                                                                0 if (90 <= x['ap_hi'] < 140) & (60 <= x['ap_lo'] < 90) else
                                                                0 if (140 <= x['ap_hi'] < 160) & (90 <= x['ap_lo'] < 100) else
                                                                0 if (x['ap_hi'] >= 160) & (x['ap_lo'] >= 100) else
                                                                1, axis=1)

        # Create cholesterol dummie variable manually
        df1['cholesterol_1'] = df1[['cholesterol']].apply(lambda x: 1 if x['cholesterol'] == 1 else 0, axis=1)
        df1['cholesterol_2'] = df1[['cholesterol']].apply(lambda x: 1 if x['cholesterol'] == 2 else 0, axis=1)
        df1['cholesterol_3'] = df1[['cholesterol']].apply(lambda x: 1 if x['cholesterol'] == 3 else 0, axis=1)

        # Create gluc dummie variable manually
        df1['gluc_1'] = df1[['gluc']].apply(lambda x: 1 if x['gluc'] == 1 else 0, axis=1)
        df1['gluc_2'] = df1[['gluc']].apply(lambda x: 1 if x['gluc'] == 2 else 0, axis=1)
        df1['gluc_3'] = df1[['gluc']].apply(lambda x: 1 if x['gluc'] == 3 else 0, axis=1)

        # Drop the columns
        df1.drop(['cholesterol', 'gluc'], axis='columns', inplace=True)

        return df1
    
    def get_prediction(self, model, original_data, test_data):
        
        # Prediction
        pred = model.predict(test_data)
        
        # Join prediction into the original data
        original_data['prediction'] = pred
        
        return original_data.to_json(orient='records')

## 8.2 API Handler

In [ ]:
import pickle, joblib
import pandas as pd
from flask import Flask, request, Response
from cardio import Cardio

# Load model
model = pickle.load(open('C:/Users/Usuario/Desktop/Data_Science/PROJETOS_DATA_SCIENCE/Projeto_Cardio_Disease/deploy/model_LGBM.pkl', 'rb'))

# Inicialize API
app = Flask(__name__)
@app.route('/cardio/predict', methods=['POST'])
def cardio_predict():
    test_json = request.get_json()
    
    if test_json:   #There is data
        if isinstance(test_json, dict):                        # unique example
            test_raw = pd.DataFrame(test_json, index=[0])  
        else:                                                  # multiple examples  
            test_raw = pd.DataFrame(test_json, columns=test_json[0].keys())
            
        # Instantiate Cardio Class
        pipeline = Cardio()
        
        # Data preparation
        df2 = pipeline.data_preparation(test_raw)
        
        # Prediction
        df_response = pipeline.get_prediction(model, test_raw, df2)
        
        return df_response
        
        
    else: 
        return Response('{}', status=200, mimetype='application/json')
        

if __name__ == '__main__':
    app.run('0.0.0.0')

## 8.3 API Tester

In [166]:
import pandas as pd
import requests

df_test = pd.read_csv('dataset/cardio_train.csv', sep=';')
df_test = df_test.sample(5)

# Convert dataframe to json
data = json.dumps(df_test.to_dict(orient='records'))

# API call
url = 'http://192.168.0.106:5000/cardio/predict'
header = {'Content-type': 'application/json'}
data = data

r = requests.post(url=url, data=data, headers=header)
print('Status Code {}.'.format(r.status_code))

Status Code 200.


In [167]:
df = pd.DataFrame(r.json(), columns=r.json()[0].keys())
df

,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,age_years,prediction
0,1,155,75.0,135,90,1,1,0,0,0,64.980556,1
1,2,170,70.0,120,80,1,1,0,0,1,44.200000,0
2,2,175,75.0,140,90,1,1,0,0,1,53.080556,1
3,1,158,88.0,150,100,1,1,0,0,1,62.438889,1
4,1,155,86.0,130,80,2,1,0,0,1,54.822222,1
